In [2]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout , Conv2D , Flatten , MaxPooling2D
from pathlib import Path
from keras.models import model_from_json
from pathlib import Path
from keras.preprocessing import image
import numpy as np


Using TensorFlow backend.


In [2]:
# Load data set
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
x_train = x_train[:200]
y_train = y_train [:200]
x_test =x_test[:20]
y_test = y_test[:20]

In [4]:
# Normalize data set to 0-to-1 range
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [5]:
# Convert class vectors to binary class matrices
# Our labels are single values from 0 to 9.
# Instead, we want each label to be an array with on element set to 1 and and the rest set to 0.
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [6]:
# Create a model and add layers
model = Sequential()
model.add(Conv2D(32,(3,3),padding='same',activation='relu',input_shape=(32,32,3)))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
#model.add(Conv2D(64,(3,3),padding='same',activation='relu'))
#model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10,activation='softmax'))

In [7]:
# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [8]:
# Print a summary of the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 7, 7, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1568)             

In [ ]:
# Train the model - Not Working in NoteBook. Works in pycharm,
'''
model.fit(
    x_train,
    y_train,
    batch_size=32,
    epochs=2,
    #validation_data=(x_test,y_test),
    shuffle=True
)

'''

Epoch 1/2


49856/50000 [============================>.] - ETA: 0s - loss: 1.1543 - accuracy: 0.5869
49888/50000 [============================>.] - ETA: 0s - loss: 1.1542 - accuracy: 0.5870
49920/50000 [============================>.] - ETA: 0s - loss: 1.1543 - accuracy: 0.5869
49952/50000 [============================>.] - ETA: 0s - loss: 1.1542 - accuracy: 0.5869
49984/50000 [============================>.] - ETA: 0s - loss: 1.1542 - accuracy: 0.5869
50000/50000 [==============================] - 242s 5ms/step - loss: 1.1543 - accuracy: 0.5869 - val_loss: 1.0201 - val_accuracy: 0.6364


In [ ]:
'''
#Save neural network Structure
model_structure = model.to_json()
f = Path("model_structure.json")
f.write_text(model_structure)

# Save model weights
model.save_weights("model_weights.h5")
'''

In [9]:

# These are the CIFAR10 class labels from the training data (in order from 0 to 9)
class_labels = [
    "Plane",
    "Car",
    "Bird",
    "Cat",
    "Deer",
    "Dog",
    "Frog",
    "Horse",
    "Boat",
    "Truck"
]

In [10]:
# Load the json file that contains the model's structure
f = Path("model_structure.json")
model_structure = f.read_text()

# Recreate the Keras model object from the json data
model = model_from_json(model_structure)

# Re-load the model's trained weights
model.load_weights("model_weights.h5")


In [11]:

# Load an image file to test, resizing it to 32x32 pixels (as required by this model)
img = image.load_img("frog.png", target_size=(32, 32))

# Convert the image to a numpy array
image_to_test = image.img_to_array(img)/255

# Add a fourth dimension to the image (since Keras expects a list of images, not a single image)
list_of_images = np.expand_dims(image_to_test, axis=0)


In [12]:
image_to_test.shape

(32, 32, 3)

In [13]:
# Make a prediction using the model
results = model.predict(list_of_images)

# Since we are only testing one image, we only need to check the first result
single_result = results[0]

# We will get a likelihood score for all 10 possible classes. Find out which class had the highest score.
most_likely_class_index = int(np.argmax(single_result))
class_likelihood = single_result[most_likely_class_index]

# Get the name of the most likely class
class_label = class_labels[most_likely_class_index]

# Print the result
print("This is image is a {} - Likelihood: {:2f}".format(class_label, class_likelihood))

This is image is a Frog - Likelihood: 0.742257
